# Fastscape (and Xarray-simlab)

A short demo of interactive model exploration using Xarray-simlab and Fastscape.

More info: https://fastscape.org/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xsimlab as xs

from fastscape.models import basic_model

## Choose and customize a fastscape LEM

In [ ]:
basic_model.visualize(show_inputs=True)

In [ ]:
from fastscape.processes import MultipleFlowRouter

model = basic_model.update_processes({'flow': MultipleFlowRouter})

In [ ]:
model.visualize(show_inputs=True)

## Run a single simulation using some reference setup

This serves as a basis for simulations that will be run further below in this notebook

Let's create a simulation setup using the `model` object create above...

In [ ]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'tstep': np.linspace(0., 1e6, 101),   # time steps in years
        'time': np.linspace(0., 1e6, 51),     # output snapshots every 2 steps 
    },
    master_clock='tstep',
    input_vars={
        'grid__shape': [201, 201],
        'grid__length': [2e4, 2e4],
        'boundary__status': ['looped', 'looped', 'fixed_value', 'fixed_value'],
        'uplift__rate': 1e-3,
        'spl__k_coef': 1e-4,
        'diffusion__diffusivity': 1e-1,
    },
    output_vars={
        'topography__elevation': 'time',
        'drainage__area': 'time',
    }
)

The simulation setup is stored into a `xarray.Dataset` object

In [ ]:
in_ds

Let's run the model...

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=model)

The simulation outputs are stored in another `xarray.Dataset`

In [ ]:
out_ds

Let's plot the results using a small utility function based on Holoviews/HvPlot. The colormap shows the logarithm of drainage area. 

In [ ]:
from xshade import plot_variable_shaded

plot_variable_shaded(out_ds, 'drainage__area', log=True, cmap=plt.cm.Blues)

## Explore the influence of flow routing slope partition exponent the on modelled topography

Let's setup and run a batch of simulations in parallel where we vary the value for the flow routing slope partition exponent.

In [ ]:
in_vars = {
    'flow__slope_exp': ("flow__slope_exp", np.arange(0., 8.))   # dimension + values
}


with model:
    out_ds2 = (
        in_ds
        .xsimlab.update_vars(input_vars=in_vars)
        .xsimlab.run(batch_dim="flow__slope_exp",
                     store="flow_runs.zarr",
                     parallel=True)
    )

In [ ]:
out_ds2

In [ ]:
plot_variable_shaded(out_ds2, 'drainage__area', batch_dim="flow__slope_exp",
                     log=True, cmap=plt.cm.Blues)

## Explore variable block uplift rates

### Time varying values

We can provide time varying values for most model parameters (excluding model inputs such as grid size).

For example, let's force the model with a sudden change (2x decrease) in uplift rate:

In [ ]:
# we can leverage the xarray `.where` function here

u_t = in_ds.uplift__rate.where(in_ds.tstep < 5e5,
                               in_ds.uplift__rate / 2)

In [ ]:
u_t.plot();

In [ ]:
in_vars = {'uplift__rate': u_t}

with model, xs.monitoring.ProgressBar():
    out_ds3 = (
        in_ds
         .xsimlab.update_vars(input_vars=in_vars)
         .xsimlab.run()
    )

In [ ]:
plot_variable_shaded(out_ds3, 'topography__elevation',
                     cmap=plt.cm.gist_earth, clim=(0, 300))

In [ ]:
(out_ds3
 .topography__elevation.sel(x=[0.5e4, 1e4, 1.5e4])
 .hvplot(x='y', y='topography__elevation',
         groupby='time', by='x',
         frame_width=400, frame_height=300)
 .redim.range(topography__elevation=(0, 300))
)

### Time + space varying values

The `uplift__rate` parameter also accepts values defined on a 2D grid, that we can combine with the time (steps) dimension.

For example, let's gradually increase the uplift towards the center of the grid:

In [ ]:
# use a 2-d gaussian
# note: we can do arithmetic operations with xarray
#       (like numpy, it supports broadcasting)

u_txy = u_t + u_t * np.exp(-((out_ds.y - 1e4)**2 / 3e7 + (out_ds.x - 1e4)**2 / 3e7))

In [ ]:
u_txy

In [ ]:
(u_txy
 .isel(tstep=[0, -1])
 .plot(col='tstep')
);

In [ ]:
(u_txy
 .sel(y=1e4)
 .isel(tstep=[0, -1])
 .plot(col='tstep')
);

In [ ]:
in_vars = {'uplift__rate': u_txy}

with model, xs.monitoring.ProgressBar():
    out_ds4 = (
        in_ds
         .xsimlab.update_vars(input_vars=in_vars)
         .xsimlab.run(check_dims="transpose")
    )

In [ ]:
plot_variable_shaded(out_ds4, 'topography__elevation',
                     cmap=plt.cm.gist_earth, clim=(0, 300))

### Batch + time + space varying values

We can add a 4th "batch" dimension for, e.g., testing various magnitudes of uplift rate:

In [ ]:
# use xarray arithmetic operations + concatenate

u_txy_batch = xr.concat([u_txy, u_txy * 2, u_txy * 3], "batch")

In [ ]:
u_txy_batch

In [ ]:
(u_txy_batch
 .isel(tstep=[0, -1])
 .plot(row='batch', col='tstep')
);

In [ ]:
in_vars = {'uplift__rate': u_txy_batch}

with model:
    out_ds5 = (
        in_ds
        .xsimlab.update_vars(input_vars=in_vars)
        .xsimlab.run(batch_dim="batch", store="uplit_runs.zarr", parallel=True)
        .assign_coords(batch=range(3))
    )

In [ ]:
plot_variable_shaded(out_ds5, 'topography__elevation',
                     batch_dim="batch",
                     cmap=plt.cm.gist_earth, clim=(0, 500))

In [ ]:
out_ds5